In [86]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import random
import os
%matplotlib inline

In [87]:
# If the names have not been downloaded
if not os.path.exists('names.txt'):
    # download the names.txt file from github
    !wget https://raw.githubusercontent.com/karpathy/makemore/master/names.txt

with open('names.txt','r') as h:
    words = [w.rstrip() for w in h.readlines()]

print(words[:8])
print(len(words))

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']
32033


In [88]:
chars = sorted(set(''.join(words)))
chars = ['.']+chars

#
# same result as two rows above as '.' is less than any [a-z] or [0-9]
# chars = sorted(set('.'.join(words))):
# print('a' > '.')
# (True)

# same:
#chars = ['.']+[*chars]

N = len(chars)
print(f"\nTotal characters: {N=}\n")

stoi = { s:i for i,s in enumerate(chars)}
itos = { i:s for s,i in stoi.items()}
print(itos)

encode = lambda c : stoi[c]
decode = lambda i : itos[i]

print(encode('a'))
print(decode(1))


Total characters: N=27

{0: '.', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z'}
1
a


In [89]:
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

random.seed(42)
random.shuffle(words)

Xtr, Ytr = build_dataset(words[:n1])
Xval, Yval = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [138]:
class Linear():
    
    def __init__(self, fan_in, fan_out, bias=True):
        self.weight = torch.randn((fan_in, fan_out), generator=g)/fan_in**0.5
        self.bias = torch.zeros(fan_out) if bias else None
#       __init__() must return None
#         return 2 <-- will cause an error
#       __call__() can return a value, still __call__() seems like an odd shortcut
        
    def __call__(self, x):
        self.out = x @ self.weight
        
        if self.bias is not None:
            self.out += self.bias
        
        return self.out
    
    def parameters(self):
        # NOTE usage of () here:
        return [self.weight] + ([] if self.bias is None else [self.bias])

    
class BatchNorm1d:
    
    def __init__(self, dim, eps=1e-5, momentum=0.1):
        self.eps = eps
        self.momentum = momentum
        self.training = True
        
        # parameters (trained with backprop)
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)
        
        # buffers (trained with a running 'momentum update')
        self.running_mean = torch.zeros(dim)
        self.running_var = torch.ones(dim)
        
        
    def __call__(self, x):
        
        if self.training:
            xmean = x.mean(dim=0, keepdim=True)
            xvar  = x.var(dim=0, keepdim=True)
        else:
            xmean = self.running_mean
            xvar = self.running_var
            
        xhat = (x - xmean)/torch.sqrt(xvar+self.eps)
        
        #
        # a typo here (had self.beta*xhat+self.beta)
        # was causing:
        #
        # the failure below 
        #
        self.out = self.gamma * xhat + self.beta
        
        if self.training:
            with torch.no_grad():
                self.running_mean = (1-self.momentum) * self.running_mean +  self.momentum * xmean
                self.running_var = (1-self.momentum) * self.running_var +  self.momentum * xvar
                
        return self.out # this is not standard    
    
    def parameters(self):
        return [self.gamma, self.beta]
    

class Tanh:
    def __call__(self, x):
        self.out = torch.tanh(x)
        return self.out
    def parameters(self):
        return []
                    

In [139]:
#
# model initialization again, but with batch norm in:
#

n_embd = 10
n_hidden = 100
vocab_size = 27
g = torch.Generator().manual_seed(2147483647)

C = torch.randn((vocab_size, n_embd), generator=g)

# a viable variant is to put batch norm after the non-linearity here:
layers = [
    Linear(n_embd * block_size, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(), 
    Linear(           n_hidden, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(), 
    Linear(           n_hidden, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(), 
    Linear(           n_hidden, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(), 
    Linear(           n_hidden, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(), 
    Linear(           n_hidden, vocab_size, bias=False), BatchNorm1d(vocab_size), 
]

with torch.no_grad():
    # make last layer less confident at initialization:
    # layers[-1].weight *= 0.1
    # Instead of weight for the last layer !!!
    layers[-1].gamma *= 0.1
    
    for layer in layers[:-1]:
        if isinstance(layer, Linear):
            layer.weight *= 1.0
            #layer.weight *= 5/3

            
parameters = [C] + [p for layer in layers for p in layer.parameters()]
print(sum([p.nelement() for p in parameters]))

for p in parameters:
    p.requires_grad = True

47024


In [140]:
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 100 # the number of neurons in the hidden layer of the MLP
g = torch.Generator().manual_seed(2147483647) # for reproducibility

C = torch.randn((vocab_size, n_embd),            generator=g)
layers = [
  Linear(n_embd * block_size, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
  Linear(           n_hidden, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
  Linear(           n_hidden, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
  Linear(           n_hidden, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
  Linear(           n_hidden, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
  Linear(           n_hidden, vocab_size, bias=False), BatchNorm1d(vocab_size),
]

with torch.no_grad():
    # last layer: make less confident
    layers[-1].gamma *= 0.1
    #layers[-1].weight *= 0.1
    # all other layers: apply gain
    for layer in layers[:-1]:
        if isinstance(layer, Linear):
            layer.weight *= 1.0 #5/3

parameters = [C] + [p for layer in layers for p in layer.parameters()]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
    p.requires_grad = True

47024


In [ ]:
Steps = 200000
batch_size=32
lossi = []

# update to data ratio:
ud = []

for step in range(Steps):

    ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
    Xb, Yb = Xtr[ix], Ytr[ix]
    
    # this is the forward pass
    emb = C[Xb]
    # emb.shape[0] is the batch size
    x = emb.view(emb.shape[0], -1)
    for layer in layers:
        x = layer(x)
    loss = F.cross_entropy(x, Yb)
    
    # backward pass
    for layer in layers:
        layer.out.retain_grad() # only needed on the debug
        
    # intialize all gradients to 0
    for p in parameters:
        p.grad = None
    
    loss.backward()
    
    # lr = 0.1 if step < Steps/2 else 0.01
    lr = 0.1 if step < 150000 else 0.01 # step learning rate decay
    
    for p in parameters:
        p.data -= lr*p.grad
    
    if step % int(Steps/1000) == 0:
        print(f"{step = },\tloss = {loss.item()}")
  
    lossi.append(loss.log10().item())
    with torch.no_grad():
        ud.append([(lr*p.grad.std() / p.data.std()).log10().item() for p in parameters])
    

step = 0,	loss = 3.284299373626709
step = 200,	loss = 2.7293994426727295
step = 400,	loss = 2.7766332626342773
step = 600,	loss = 2.739665985107422
step = 800,	loss = 2.6246838569641113
step = 1000,	loss = 2.535635471343994
step = 1200,	loss = 2.358489990234375
step = 1400,	loss = 2.239806890487671
step = 1600,	loss = 2.61871337890625
step = 1800,	loss = 2.7463929653167725
step = 2000,	loss = 2.352499485015869
step = 2200,	loss = 1.9942113161087036
step = 2400,	loss = 2.388838529586792
step = 2600,	loss = 2.5457427501678467
step = 2800,	loss = 2.485955238342285
step = 3000,	loss = 2.556744337081909
step = 3200,	loss = 2.32403564453125
step = 3400,	loss = 2.346729278564453
step = 3600,	loss = 2.4430980682373047
step = 3800,	loss = 2.5304322242736816
step = 4000,	loss = 2.2273709774017334
step = 4200,	loss = 2.155994415283203
step = 4400,	loss = 2.457611083984375
step = 4600,	loss = 2.3537795543670654
step = 4800,	loss = 2.2429792881011963
step = 5000,	loss = 2.2977664470672607
step = 52